In [1]:
dataurlspath = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/bibliographic/2014/'
dataurls =[
'ipgb20140107_wk01.zip',
'ipgb20140114_wk02.zip',
'ipgb20140121_wk03.zip',
'ipgb20140128_wk04.zip',
'ipgb20140204_wk05.zip',
'ipgb20140211_wk06.zip',
'ipgb20140218_wk07.zip',
'ipgb20140225_wk08.zip',
'ipgb20140304_wk09.zip',
'ipgb20140311_wk10.zip',
'ipgb20140318_wk11.zip',
'ipgb20140325_wk12.zip',
'ipgb20140401_wk13.zip',
'ipgb20140408_wk14.zip',
'ipgb20140415_wk15.zip',
'ipgb20140422_wk16.zip',
'ipgb20140429_wk17.zip',
'ipgb20140506_wk18.zip',
'ipgb20140513_wk19.zip',
'ipgb20140520_wk20.zip',
'ipgb20140527_wk21.zip',
'ipgb20140603_wk22.zip',
'ipgb20140610_wk23.zip',
'ipgb20140617_wk24.zip',
'ipgb20140624_wk25.zip',
'ipgb20140701_wk26.zip',
'ipgb20140708_wk27.zip',
'ipgb20140715_wk28.zip',
'ipgb20140722_wk29.zip',
'ipgb20140729_wk30.zip',
'ipgb20140805_wk31.zip',
'ipgb20140812_wk32.zip',
'ipgb20140819_wk33.zip',
'ipgb20140826_wk34.zip',
'ipgb20140902_wk35.zip',
'ipgb20140909_wk36.zip',
'ipgb20140916_wk37.zip',
'ipgb20140923_wk38.zip',
'ipgb20140930_wk39.zip',
'ipgb20141007_wk40.zip',
'ipgb20141014_wk41.zip',
'ipgb20141021_wk42.zip',
'ipgb20141028_wk43.zip',
'ipgb20141104_wk44.zip',
'ipgb20141111_wk45.zip',
'ipgb20141118_wk46.zip',
'ipgb20141125_wk47.zip',
'ipgb20141202_wk48.zip',
'ipgb20141209_wk49.zip',
'ipgb20141216_wk50.zip',
'ipgb20141223_wk51.zip',
'ipgb20141230_wk52.zip'
]

In [2]:
def xmltocsv(inputfilename, filename):
    
    #fName = '../data/priorartoutput/priorart_data_' + filename[:-4] + '.csv'
    #fw = open(fName, 'w')
    fw = open(filename, 'a')
    w = csv.writer(fw, delimiter = ',')

    dfilepath = inputfilename
    dfilepathtmp = dfilepath + 'tmp'
    !grep -v '<?xml version="1.0" encoding="UTF-8"?>' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -e '/^<!DOCTYPE/d' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -i '1 i\<items>' $dfilepath
    !sleep 5
    #!sed -i -e '$a</items>' $dfilepath
    !sed -i '$ a \</items>' $dfilepath
    
    pub_doc_number = ''
    appl_doc_number = ''
    appl_country = ''
    invention_title = ''
    abstract = ''
    for event, element in etree.iterparse(inputfilename, tag="us-patent-grant"):
        for child in element:
            
            if child.tag == 'us-bibliographic-data-grant':
                for child1 in child:
                    if child1.tag == 'publication-reference':
                        if child1[0][1].tag == 'doc-number':
                            pub_doc_number = child1[0][1].text
                    
                    if child1.tag == 'application-reference':
                        if child1[0][1].tag == 'doc-number':
                            appl_country = child1[0][0].text 
                            appl_doc_number = child1[0][1].text 
                            
                    if child1.tag == 'us-references-cited':
                        ref_doc_numbers = ''
                        for child2 in child1:
                            if child2[1].text == 'cited by examiner': #'category':
                                #<category>cited by examiner</category>
                                if child2[0].tag == 'patcit':
                                    if child2[0][0][2].text == 'A':
                                        if ref_doc_numbers == '':
                                            ref_doc_numbers = child2[0][0][1].text
                                        else:
                                            ref_doc_numbers = ref_doc_numbers + ',' + child2[0][0][1].text
                                    
                        if ref_doc_numbers != '':              
                            w.writerow([pub_doc_number, appl_doc_number, ref_doc_numbers])
                            pub_doc_number = ''
                            appl_doc_number = ''
                            ref_doc_numbers = ''
                            break

                            #print(appl_doc_number, ref_doc_numbers)
  
        element.clear()
    fw.close() 

In [3]:
import csv
import time
import os
import zipfile
import lxml.etree as etree, lxml.html
import string


indir = '../data/priorartdata'
loaddir = '../data/priorartloadzone'
loaddelete = '../data/priorartloadzone/*'

filename = '../data/priorartoutput/priorart_data_2014.csv'

for i in range(0,len(dataurls)):
    print('Downloading file:' + str(i))
    #down load the file and 
    dataurl = dataurlspath + dataurls[i]
    !curl -L $dataurl -o '../data/priorartdata/datafile.zip'
    
    for path, subdirs, files in os.walk(indir):

        for f in files:
            filepath = os.path.join(path, f)
            print(filepath)
            !rm -r $loaddelete
            zip_ref = zipfile.ZipFile(filepath, 'r')
            zip_ref.extractall('../data/priorartloadzone')
            zip_ref.close()

            datadir = '../data/priorartloadzone'
            for dpath, dsubdirs, dfiles in os.walk(datadir):
                for df in dfiles:
                    if df.endswith('xml'):
                        dfilepath = os.path.join(dpath, df)
                        print('Starting file: ',dfilepath )
                        #!grep -rl "us-references-cited" /home/ubuntu/data/fulltextloadzone
                        #xmltocsv(dfilepath,dataurls[i])
                        xmltocsv(dfilepath,filename)
                        print('Loaded file: ', dfilepath )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.2M  100 12.2M    0     0  13.8M      0 --:--:-- --:--:-- --:--:-- 13.8M
../data/priorartdata/datafile.zip
Starting file:  ../data/priorartloadzone/ipgb20140107.xml
Loaded file:  ../data/priorartloadzone/ipgb20140107.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9118k  100 9118k    0     0  11.5M      0 --:--:-- --:--:-- --:--:-- 11.9M
../data/priorartdata/datafile.zip
Starting file:  ../data/priorartloadzone/ipgb20140114.xml
Loaded file:  ../data/priorartloadzone/ipgb20140114.xml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9353k  100 9353k    0     0  11.7M      0 --:--:-- --:--:-- --:--:--

In [2]:
#!aws s3 cp /home/ubuntu/data/priorartoutput/priorart_data_2014.csv  s3://ncw210data/PatentData/priorart_data_2014.csv

In [12]:
#https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows
import pandas as pd
df_priorart = pd.read_csv('/home/ubuntu/data/priorartoutput/priorart_data_2014.csv',
                      names=['publication_number','application_number','priorart_numbers'])

df_priorart_split = pd.DataFrame(df_priorart.priorart_numbers.str.split(',').tolist(), index=df_priorart.application_number).stack()
df_priorart_split = df_priorart_split.reset_index()[[0, 'application_number']] # var1 variable is currently labeled 0
df_priorart_split.columns = ['priorart_number', 'application_number'] # renaming var1

df_priorart_split.to_csv('/home/ubuntu/data/priorartoutput/priorart_data_split_2014.csv',index=False, sep=',') 

In [1]:
#copy files to S3
#!aws s3 cp /home/ubuntu/data/priorartoutput/priorart_data_split_2014.csv  s3://ncw210data/PatentData/priorart_data_split_2014.csv

In [24]:
#!grep -r "6007845" /home/ubuntu/data/applicationdata/application_data.csv
#!grep -r "Poynton" /home/ubuntu/data/applicationdata/application_data.csv 
